<a href="https://colab.research.google.com/github/JakubChmielewskiRepo/wstep-do-uczenia-maszynowego-175ic/blob/master/lab5/lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# !pip install spacy --upgrade
# !spacy download pl_core_news_sm
# !pip install textacy --upgrade

import spacy
import pl_core_news_sm 
import re
from spacy.tokenizer import Tokenizer
from collections import Counter
from spacy import displacy
from spacy.matcher import Matcher
import textacy
import warnings

warnings.filterwarnings("ignore")

nlp = pl_core_news_sm.load()


#Czytanie Stringa
introduction_text = ('This tutorial is abałt Natural Language Processing in Spacy.')
introduction_doc = nlp(introduction_text)
# Extract tokens for the given doc
print ([token.text for token in introduction_doc])




['This', 'tutorial', 'is', 'abałt', 'Natural', 'Language', 'Processing', 'in', 'Spacy', '.']


In [44]:
#Czytanie pliku tekstowego
file_name = 'introduction.txt'
introduction_file_text = ('Wczoraj o godzinie Jakuba Chmielewskiego osiemnastej odbyło się zebranie Stowarzyszenia Bełchatowskich Rodzin, na którym omawiano drażniący problem lanego poniedziałku. Wysunięto także propozycję, aby na ulicę wyszły prewencyjnie liczniejsze niż zwykle patrole policji i straży miejskiej. Rodziny zaapelowały do młodzieży, żeby ta nie przesadzała z wodą. Tymczasem sama młodzież mówi, że nic złego się nigdy nie dzieje. „My się oblewamy między sobą, serio. Na pewno do żadnego dorosłego kolesia z butlami nie przylecimy. Nie wiem, czemu wszyscy tak się tego poniedziałku boją!”. Koledzy mojego rozmówcy tylko przytakują głowami.')
introduction_file_doc = nlp(introduction_file_text)
# Extract tokens for the given doc
print ([token.text for token in introduction_file_doc])

['Wczoraj', 'o', 'godzinie', 'Jakuba', 'Chmielewskiego', 'osiemnastej', 'odbyło', 'się', 'zebranie', 'Stowarzyszenia', 'Bełchatowskich', 'Rodzin', ',', 'na', 'którym', 'omawiano', 'drażniący', 'problem', 'lanego', 'poniedziałku', '.', 'Wysunięto', 'także', 'propozycję', ',', 'aby', 'na', 'ulicę', 'wyszły', 'prewencyjnie', 'liczniejsze', 'niż', 'zwykle', 'patrole', 'policji', 'i', 'straży', 'miejskiej', '.', 'Rodziny', 'zaapelowały', 'do', 'młodzieży', ',', 'żeby', 'ta', 'nie', 'przesadzała', 'z', 'wodą', '.', 'Tymczasem', 'sama', 'młodzież', 'mówi', ',', 'że', 'nic', 'złego', 'się', 'nigdy', 'nie', 'dzieje', '.', '„', 'My', 'się', 'oblewamy', 'między', 'sobą', ',', 'serio', '.', 'Na', 'pewno', 'do', 'żadnego', 'dorosłego', 'kolesia', 'z', 'butlami', 'nie', 'przylecimy', '.', 'Nie', 'wiem', ',', 'czemu', 'wszyscy', 'tak', 'się', 'tego', 'poniedziałku', 'boją', '!', '”', '.', 'Koledzy', 'mojego', 'rozmówcy', 'tylko', 'przytakują', 'głowami', '.']


In [29]:
#Wypisywanie zdań z tekstu
about_text = ('Gus Proto is a Python developer currently working for a London-based Fintech company. He is interested in learning Natural Language Processing.')
about_doc = nlp(about_text)
sentences = list(about_doc.sents)
for sentence in sentences:
  print(sentence)
  

Gus Proto is a Python developer currently working for a London-based Fintech company.
He is interested in learning Natural Language Processing.


In [30]:
#Porównanie wypisywania zdań z customowym znacznikiem, oraz ze znacznikiem domyślnym
def set_custom_boundaries(doc):
# Adds support to use `...` as the delimiter for sentence detection
  for token in doc[:-1]:
    if token.text == '...':
      doc[token.i+1].is_sent_start = True
  return doc

ellipsis_text = ('Gus, can you, ... never mind, I forgot what I was saying. So, do you think we should ...')
# Load a new model instance
custom_nlp = spacy.load('en_core_web_sm')
custom_nlp.add_pipe(set_custom_boundaries, before='parser')
custom_ellipsis_doc = custom_nlp(ellipsis_text)
custom_ellipsis_sentences = list(custom_ellipsis_doc.sents)
for sentence in custom_ellipsis_sentences:
  print(sentence)
# Sentence Detection with no customization
ellipsis_doc = nlp(ellipsis_text)
ellipsis_sentences = list(ellipsis_doc.sents)
for sentence in ellipsis_sentences:
  print(sentence)

Gus, can you, ...
never mind, I forgot what I was saying.
So, do you think we should ...
Gus, can you, ... never mind, I forgot what I was saying.
So, do you think we should ...


In [14]:
#Tokenizacja
# for token in introduction_file_doc:
#   print (token, token.idx)

# for token in introduction_file_doc:
#   print (token, token.idx, token.text_with_ws,token.is_alpha, token.is_punct, token.is_space,token.shape_, token.is_stop)


prefix_re = spacy.util.compile_prefix_regex(custom_nlp.Defaults.prefixes)
suffix_re = spacy.util.compile_suffix_regex(custom_nlp.Defaults.suffixes)
infix_re = re.compile(r'''[-~]''')
def customize_tokenizer(nlp):
# Adds support to use `-` as the delimiter for tokenization
 return Tokenizer(nlp.vocab, prefix_search=prefix_re.search,suffix_search=suffix_re.search,infix_finditer=infix_re.finditer,token_match=None)

custom_nlp.tokenizer = customize_tokenizer(custom_nlp)
custom_tokenizer_about_doc = custom_nlp(introduction_text)
print([token.text for token in custom_tokenizer_about_doc])

['This', 'tutorial', 'is', 'abałt', 'Natural', 'Language', 'Processing', 'in', 'Spacy', '.']


In [31]:
#Słowa pomijalne
import spacy
spacy_stopwords = spacy.lang.pl.stop_words.STOP_WORDS
len(spacy_stopwords)
326
for stop_word in list(spacy_stopwords)[:10]:
  print(stop_word)

#Usuwanie pomijalnych słów
for token in introduction_file_doc:
  if not token.is_stop:
    print (token)

about_no_stopword_doc = [token for token in introduction_file_doc if not token.is_stop]
print (about_no_stopword_doc)

ty
was
wszystko
podczas
one
przed
tego
mam
mogą
je
Wczoraj
godzinie
osiemnastej
odbyło
zebranie
Stowarzyszenia
Bełchatowskich
Rodzin
,
omawiano
drażniący
problem
lanego
poniedziałku
.
Wysunięto
propozycję
,
ulicę
wyszły
prewencyjnie
liczniejsze
zwykle
patrole
policji
straży
miejskiej
.
Rodziny
zaapelowały
młodzieży
,
przesadzała
wodą
.
Tymczasem
młodzież
mówi
,
złego
dzieje
.
„
oblewamy
,
serio
.
pewno
żadnego
dorosłego
kolesia
butlami
przylecimy
.
wiem
,
poniedziałku
boją
!
”
.
Koledzy
mojego
rozmówcy
przytakują
głowami
.
[Wczoraj, godzinie, osiemnastej, odbyło, zebranie, Stowarzyszenia, Bełchatowskich, Rodzin, ,, omawiano, drażniący, problem, lanego, poniedziałku, ., Wysunięto, propozycję, ,, ulicę, wyszły, prewencyjnie, liczniejsze, zwykle, patrole, policji, straży, miejskiej, ., Rodziny, zaapelowały, młodzieży, ,, przesadzała, wodą, ., Tymczasem, młodzież, mówi, ,, złego, dzieje, ., „, oblewamy, ,, serio, ., pewno, żadnego, dorosłego, kolesia, butlami, przylecimy, ., wiem, ,, ponie

In [32]:
#Lemmatyzacja - grupowanie słów
conference_help_text = ('Gus is helping organize a developer conference on Applications of Natural Language Processing. He keeps organizing local Python meetups and several internal talks at his workplace.')
conference_help_doc = nlp(conference_help_text)
for token in conference_help_doc:
  print (token, token.lemma_)

  

Gus gus
is is
helping helping
organize organize
a a
developer developer
conference conference
on on
Applications applications
of of
Natural natural
Language language
Processing processing
. .
He he
keeps keeps
organizing organizing
local local
Python python
meetups meetups
and and
several several
internal internal
talks talks
at at
his his
workplace workplace
. .


In [43]:
#Słowa powtarzające się i unikalne
from collections import Counter
complete_text = ('W ubiegłym roku śmigus- dyngus okazał się wyjątkowo mokry Jakub Chmielewski i wyjątkowo brzemienny w skutkach. W wyniku „niewinnych” igraszek z wodą doszło do wypadku samochodowego, w wyniku którego jedna osoba doznała poważnych obrażeń kręgosłupa. Było to na szczęście jedyne tak tragiczne wydarzenie owego dnia.')


complete_doc = nlp(complete_text)
# Remove stop words and punctuation symbols
words = [token.text for token in complete_doc
  if not token.is_stop and not token.is_punct]
word_freq = Counter(words)
# 5 commonly occurring words with their frequencies
common_words = word_freq.most_common(5)
print (common_words)

# Unique words
unique_words = [word for (word, freq) in word_freq.items() if freq == 1]
print (unique_words)

[('wyjątkowo', 2), ('wyniku', 2), ('ubiegłym', 1), ('śmigus', 1), ('dyngus', 1)]
['ubiegłym', 'śmigus', 'dyngus', 'okazał', 'mokry', 'Jakub', 'Chmielewski', 'brzemienny', 'skutkach', 'niewinnych', 'igraszek', 'wodą', 'doszło', 'wypadku', 'samochodowego', 'osoba', 'doznała', 'poważnych', 'obrażeń', 'kręgosłupa', 'szczęście', 'jedyne', 'tragiczne', 'wydarzenie', 'owego', 'dnia']


In [34]:
#Rozpoznawanie częsci mowy
for token in complete_doc:
  print (token, token.tag_, token.pos_, spacy.explain(token.tag_))

W PREP ADP None
ubiegłym ADJ ADJ adjective
roku SUBST NOUN None
śmigus SUBST NOUN None
- INTERP PUNCT None
dyngus SUBST NOUN None
okazał PRAET VERB None
się QUB PART None
wyjątkowo ADV ADV adverb
mokry ADJ ADJ adjective
i CONJ CCONJ conjunction
wyjątkowo ADV ADV adverb
brzemienny ADJ ADJ adjective
w PREP ADP None
skutkach SUBST NOUN None
. INTERP PUNCT None
W PREP ADP None
wyniku SUBST NOUN None
„ INTERP PUNCT None
niewinnych ADJ ADJ adjective
” INTERP PUNCT None
igraszek SUBST NOUN None
z PREP ADP None
wodą SUBST NOUN None
doszło PRAET VERB None
do PREP ADP None
wypadku SUBST NOUN None
samochodowego ADJ ADJ adjective
, INTERP PUNCT None
w PREP ADP None
wyniku SUBST NOUN None
którego ADJ ADJ adjective
jedna ADJ ADJ adjective
osoba SUBST NOUN None
doznała PRAET VERB None
poważnych ADJ ADJ adjective
obrażeń SUBST NOUN None
kręgosłupa SUBST NOUN None
. INTERP PUNCT None
Było PRAET VERB None
to SUBST NOUN None
na PREP ADP None
szczęście SUBST NOUN None
jedyne ADJ ADJ adjective
tak ADV ADV 

In [36]:
#Filtrowanie słów według kategorii:
nouns = []
adjectives = []
for token in introduction_file_doc:
    if token.pos_ == 'NOUN':
        nouns.append(token)
    if token.pos_ == 'ADJ':
        adjectives.append(token)

print(f'Rzeczowniki: {nouns}')
print(f'Przymiotniki: {adjectives}')

Rzeczowniki: [godzinie, zebranie, Stowarzyszenia, Rodzin, problem, poniedziałku, propozycję, ulicę, prewencyjnie, patrole, policji, straży, Rodziny, młodzieży, wodą, młodzież, nic, kolesia, butlami, poniedziałku, Koledzy, rozmówcy, głowami]
Przymiotniki: [osiemnastej, Bełchatowskich, którym, lanego, liczniejsze, miejskiej, ta, sama, złego, żadnego, dorosłego, wszyscy, tego, mojego]


In [38]:
#Wizualizacja displaCy
about_displaCy_text = "To jest testowy tekst po polsku, nic on nie znaczy i jest bez sensu"
about_displaCy_doc = nlp(about_displaCy_text)
displacy.serve(about_displaCy_doc, style='dep')


Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [39]:
#Funkcje przetwarzania wstępnego
def is_token_allowed(token):
    if not token or not token.string.strip() or token.is_stop or token.is_punct:
        return False
    return True

def preprocess_token(token):
    return token.lemma_.strip().lower()

complete_filtered_tokens = [preprocess_token(token)for token in complete_doc if is_token_allowed(token)]
print(complete_filtered_tokens)

['ubiegły', 'śmigus', 'dyngus', 'okazać', 'wyjątkowo', 'mokry', 'wyjątkowo', 'brzemienny', 'skutek', 'wynik', 'winny', 'igraszka', 'woda', 'dojść', 'wypadek', 'samochodowy', 'wynik', 'osoba', 'doznać', 'poważny', 'obrażenie', 'kręgosłup', 'szczęście', 'jedyny', 'tragiczny', 'wydarzenie', 'ów', 'dzień']


In [47]:
#Ekstrakcja numeru telefonu 

phone_number=("cos tam 222 333 444 cos tam")

def extract_phone_number(nlp_doc):
	pattern = [{"SHAPE": "ddd"}, {"SHAPE": "ddd"},{"SHAPE": "ddd"}]
	matcher.add('PHONE_NUMBER', None, pattern)
	matches = matcher(nlp_doc)
	for match_id, start, end in matches:
		span = nlp_doc[start:end]
		return span.text

conference_org_doc = nlp(phone_number)
extract_phone_number(conference_org_doc)

'222 333 444'

In [48]:
#Struktura zdania

text = ('Daj mi tę noc, te jedną noc')
text_doc = nlp(text)
for token in text_doc:
  print (token.text, token.tag_, token.head.text, token.dep_)
displacy.serve(text_doc, style='dep')

Daj IMPT Daj ROOT
mi PPRON12 Daj iobj
tę ADJ noc det
noc SUBST Daj obj
, INTERP noc punct
te ADJ noc det
jedną ADJ noc amod
noc SUBST Daj obl

Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [52]:
#Drzewa

one_line_about_text = ('Chciałbym być marynarzem chciałbym mieć tatuaże')
one_line_about_doc = nlp(one_line_about_text)

# Wyświetlenie sąsiadów słowa "chciałbym"
print([token.text for token in one_line_about_doc[3].children])
print (one_line_about_doc[3].nbor(-1))
print (one_line_about_doc[3].nbor())
#Poddrzewo słowa "chciałbym"
print (list(one_line_about_doc[3].subtree))

['być', 'marynarzem', 'mieć']
marynarzem
mieć
[być, marynarzem, chciałbym, mieć, tatuaże]


In [57]:
#Wykrywanie fraz rzeczownikowych
conference_text = ('Jedzenie korzenie samouwielbienie')
conference_doc = nlp(conference_text)
for chunk in conference_doc.noun_chunks:
  print (chunk)

In [62]:
about_talk_text=('Komiczny komik terroryzował miasto mając jedynie lwa i płonącą obręcz')
pattern = [{"POS": "VERB", "OP": "*"},{"POS": "VERB", "OP": "+"}]
about_talk_doc = textacy.make_spacy_doc(about_talk_text, nlp)
verb_phrases = textacy.extract.matches(about_talk_doc, pattern)
for chunk in verb_phrases:
    print(chunk.text)

for chunk in about_talk_doc.noun_chunks:
    print (chunk)


terroryzował
mając
płonącą


In [64]:
#Ukrywanie imienia i nazwiska
survey_text =('Tatiana Trąbalska wcale nie ma trąby jak twierdzi nasz specjalny wysłannik Tomasz Beztrąby')
def replace_person_names(token):
    if token.ent_iob != 0 and token.ent_type_ == 'persName':
        return 'XYZ '
    return token.string

def redact_names(nlp_doc):
    for ent in nlp_doc.ents:
        ent.merge()
    tokens = map(replace_person_names, nlp_doc)
    return ''.join(tokens)

survey_doc = nlp(survey_text)
print(redact_names(survey_doc))
print(survey_text)

XYZ wcale nie ma trąby jak twierdzi nasz specjalny wysłannik XYZ 
Tatiana Trąbalska wcale nie ma trąby jak twierdzi nasz specjalny wysłannik Tomasz Beztrąby
